## Select / download video

In [ ]:
!pip install yt-dlp moviepy openai-whisper==20230124 whisper-timestamped

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.6/171.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.7/764.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230124-py3-none-any.whl size=1179308 sha256=b94174083a9598e9a5df468eddd348919dc7e3afb91551dfcc7951139613b7

In [ ]:
import os
import json
import numpy as np
import whisper_timestamped
from moviepy.editor import VideoFileClip
import yt_dlp

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



  if event.key is 'enter':



In [ ]:
def download_audio_from_youtube(url):
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': '%(id)s.%(ext)s',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
                'preferredquality': '192',
            }],
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            audio_file = f"{info_dict['id']}.wav"
            return audio_file

    except Exception as e:
        return str(e)

In [ ]:
def extract_audio_from_video(video_file):
    try:
        audio_file = "extracted_audio.wav"
        video_clip = VideoFileClip(video_file)
        video_clip.audio.write_audiofile(audio_file, codec='pcm_s16le', ffmpeg_params=["-ac", "1", "-ar", "16000"])
        return audio_file

    except Exception as e:
        return str(e)

In [ ]:
def process_audio(audio_file):
    try:
        print("Processing audio file:", os.path.abspath(audio_file))
        if not os.path.exists(audio_file):
            return "Audio file does not exist."

        audio = whisper_timestamped.load_audio(audio_file)
        audio = audio / np.max(np.abs(audio))
        model = whisper_timestamped.load_model("base", device="cpu")
        result = whisper_timestamped.transcribe_timestamped(model, audio, language="en")

        # Save results to SRT file
        writer = whisper_timestamped.utils.get_writer("srt", ".")
        writer(result, "output")
        print(f"SRT file saved: transcription.srt")

        # Save results to JSON file
        json_output_file = "output.json"
        with open(json_output_file, 'w', encoding='utf-8') as json_file:
            json.dump(result, json_file, indent=2, ensure_ascii=False)
        print(f"JSON file saved: {json_output_file}")

        return "Transcription completed"

    except Exception as e:
        return str(e)

Use Youtube link

In [ ]:
youtube_url = 'https://www.youtube.com/watch?v=XdqPpRRewrE'  # Change this to YouTube URL
audio_file = download_audio_from_youtube(youtube_url)
if audio_file:
    result = process_audio(audio_file)
    print(result)

[youtube] Extracting URL: https://www.youtube.com/watch?v=XdqPpRRewrE
[youtube] XdqPpRRewrE: Downloading webpage
[youtube] XdqPpRRewrE: Downloading ios player API JSON
[youtube] XdqPpRRewrE: Downloading mweb player API JSON
[youtube] XdqPpRRewrE: Downloading m3u8 information
[info] XdqPpRRewrE: Downloading 1 format(s): 251
[download] Destination: XdqPpRRewrE.webm
[download] 100% of   15.98MiB in 00:00:00 at 27.27MiB/s  
[ExtractAudio] Destination: XdqPpRRewrE.wav
Deleting original file XdqPpRRewrE.webm (pass -k to keep)
Processing audio file: /content/XdqPpRRewrE.wav


  checkpoint = torch.load(fp, map_location=device)

  warnings.warn(

100%|██████████| 111395/111395 [07:10<00:00, 258.53frames/s]


SRT file saved: transcription.srt
JSON file saved: output.json
Transcription completed


Use local path

In [ ]:
video_file = 'path_to_your_video_file.mp4'  # Change this to video file path
audio_file = extract_audio_from_video(video_file)
if audio_file:
    result = process_audio(audio_file)
    print(result)

Processing audio file: /content/MoviePy error: the file path_to_your_video_file.mp4 could not be found!
Please check that you entered the correct path.
Audio file does not exist.


## Extract text

In [ ]:
import json

In [ ]:
file_path = 'output.json'

with open(file_path, 'r') as file:
    data = json.load(file)

print(data["text"])

 Okay, let's talk about why we make bad decisions. So if you're like me, then you've probably been in that situation where it's the end of the day or it's the morning and you're thinking like, okay, I'm going to make a solid plan for myself. Today, I'm going to be disciplined. I'm going to come home from work and I'm going to get all these errands done and I'm going to treat myself really well. I'm going to make a nice, healthy, proper meal. I'm going to go to the gym afterwards. I'm going to exercise and then maybe when I come home, I'll like start reading a book or something. I'll do something a little different. Something that makes me feel like I'm going to advance my life in a new, positive direction. This is our real life volition and it's there to lead you on your way to making good life decisions. But this doesn't always work out that way. Sometimes you get to the end of the day and you realize that you haven't been very disciplined. At all. A lot of those errands are still inc

In [ ]:
# Extract the desired features
word_timings = []
for segment in data['segments']:
    for word in segment['words']:
        word_timing = {
            'text': word['text'],
            'start': word['start'],
            'end': word['end']
        }
        word_timings.append(word_timing)

# Print the total number of words extracted
print(f"Total words extracted: {len(word_timings)}")

# Print the last few entries to verify
print("Last 5 entries:")
for entry in word_timings[-5:]:
    print(entry)

# Optionally, save to a new JSON file
with open('extracted_word_timings.json', 'w') as outfile:
    json.dump(word_timings, outfile, indent=2)

print("Extracted data saved to 'extracted_word_timings.json'")

Total words extracted: 3009
Last 5 entries:
{'text': 'own', 'start': 1096.54, 'end': 1096.68}
{'text': 'good', 'start': 1096.68, 'end': 1096.9}
{'text': 'decisions', 'start': 1096.9, 'end': 1097.32}
{'text': 'right', 'start': 1097.32, 'end': 1097.9}
{'text': 'now.', 'start': 1097.9, 'end': 1098.14}
Extracted data saved to 'extracted_word_timings.json'


Segament with .

In [ ]:
import re

In [ ]:
sentences = re.split(r'[.?!]', data["text"])
sentences = [sentence.strip() for sentence in sentences if sentence]

In [ ]:
for i in range(len(sentences)):
  print(sentences[i])

Okay, let's talk about why we make bad decisions
So if you're like me, then you've probably been in that situation where it's the end of the day or it's the morning and you're thinking like, okay, I'm going to make a solid plan for myself
Today, I'm going to be disciplined
I'm going to come home from work and I'm going to get all these errands done and I'm going to treat myself really well
I'm going to make a nice, healthy, proper meal
I'm going to go to the gym afterwards
I'm going to exercise and then maybe when I come home, I'll like start reading a book or something
I'll do something a little different
Something that makes me feel like I'm going to advance my life in a new, positive direction
This is our real life volition and it's there to lead you on your way to making good life decisions
But this doesn't always work out that way
Sometimes you get to the end of the day and you realize that you haven't been very disciplined
At all
A lot of those errands are still incomplete or hal

In [ ]:
cleaned_text = data["text"].replace(",", "").replace(".", "").replace("!", "").replace("?", "")

In [ ]:
print(cleaned_text)

 Okay let's talk about why we make bad decisions So if you're like me then you've probably been in that situation where it's the end of the day or it's the morning and you're thinking like okay I'm going to make a solid plan for myself Today I'm going to be disciplined I'm going to come home from work and I'm going to get all these errands done and I'm going to treat myself really well I'm going to make a nice healthy proper meal I'm going to go to the gym afterwards I'm going to exercise and then maybe when I come home I'll like start reading a book or something I'll do something a little different Something that makes me feel like I'm going to advance my life in a new positive direction This is our real life volition and it's there to lead you on your way to making good life decisions But this doesn't always work out that way Sometimes you get to the end of the day and you realize that you haven't been very disciplined At all A lot of those errands are still incomplete or half-starte

## Sentence segamentation

In [ ]:
# Import the Python SDK
import google.generativeai as genai

genai.configure(api_key='YOUR_KEY')

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
text = data["text"]

In [ ]:
# Segmentation function
def segment_text_gemini(model, text, max_length=50):
    prompt = f"""
    These resultes will be used as captions for video.
    Try to make sentences grammatically complete.
    Do not change any words from the input text.
    Segment the following text into new sentences with each sentence no longer than {max_length} characters: \n\n{text}
    """

    # Send request to model
    response = model.generate_content(prompt)
    return response

# Get segmented text
segmented_output = segment_text_gemini(model, text)

In [ ]:
print(segmented_output.text)

In [ ]:
"""
Okay let's talk about why we make bad decisions
So if you're like me then you've probably been in that situation
where it's the end of the day or it's the morning
and you're thinking like okay I'm going to make a solid plan for myself
Today I'm going to be disciplined
I'm going to come home from work and I'm going to get all these errands done
and I'm going to treat myself really well
I'm going to make a nice healthy proper meal
I'm going to go to the gym afterwards
I'm going to exercise and th
"""